# Introduction to RoutingZoo

#### Import libraries

In [1]:
import sys
import os
from pathlib import Path

RoutingZoo = str(Path.home() / "Documents/Simulator_human_behaviour")

sys.path.append(RoutingZoo)

import data_analysis as da
import utilities_RZ as URZ


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))

from RouteRL.environment.environment import TrafficEnvironment
from RouteRL.keychain import Keychain as kc


from RouteRL.utilities import check_device
from RouteRL.utilities import get_params
from RouteRL.utilities import set_seeds

Simulator path resolved to: /Users/zoltanvarga/Documents/Simulator_human_behaviour


In [2]:
check_device()
set_seeds()
params = get_params("params.json")

[INFO] Running on device: cpu


#### Environment initialization

In [3]:
kwargs = {
        "learning_type" : 'markow',
        "gamma_c" : 0.2,
        "gamma_u" : 0,
        "remember" : 5,
        "greedy" : 0,
        "noise_alpha" : 0,
        "noise_taste" : 0.8,
        "noise_random" : 0.2,
        "network":'grid1',
        "demand":params[kc.AGENT_GEN][kc.NUM_AGENTS]
        }

In [4]:
env = TrafficEnvironment(params[kc.RUNNER], params[kc.ENVIRONMENT], params[kc.SIMULATOR], params[kc.AGENT_GEN], params[kc.AGENTS], params[kc.PLOTTER],**kwargs)

[CONFIRMED] Environment variable exists: SUMO_HOME
[SUCCESS] Added module directory: /opt/homebrew/opt/sumo/share/sumo/tools


In [5]:
print("Number of total agents is: ", len(env.all_agents), "\n")
print("Agents are: ", env.all_agents, "\n")
print("Number of human agents is: ", len(env.human_agents), "\n")
print("Number of machine agents (autonomous vehicles) is: ", len(env.machine_agents), "\n")

Number of total agents is:  1000 

Agents are:  [<agent.HumanAgent object at 0x1055bdeb0>, <agent.HumanAgent object at 0x1056bf2e0>, <agent.HumanAgent object at 0x12ee1b190>, <agent.HumanAgent object at 0x10566dcd0>, <agent.HumanAgent object at 0x1056bf7f0>, <agent.HumanAgent object at 0x1056bf820>, <agent.HumanAgent object at 0x1056bf190>, <agent.HumanAgent object at 0x1056bfc70>, <agent.HumanAgent object at 0x1056bf460>, <agent.HumanAgent object at 0x12ee1bd90>, <agent.HumanAgent object at 0x12ee1b9a0>, <agent.HumanAgent object at 0x1056bf490>, <agent.HumanAgent object at 0x12ee1ba30>, <agent.HumanAgent object at 0x105687df0>, <agent.HumanAgent object at 0x105687b20>, <agent.HumanAgent object at 0x105687280>, <agent.HumanAgent object at 0x105687b80>, <agent.HumanAgent object at 0x105687580>, <agent.HumanAgent object at 0x105687340>, <agent.HumanAgent object at 0x1056878b0>, <agent.HumanAgent object at 0x105687f70>, <agent.HumanAgent object at 0x1056bf310>, <agent.HumanAgent object at

#### Human learning

In [6]:
num_episodes =  3

env.start()

for episode in range(num_episodes):
    
    env.step()
    print("1", env.simulator.det_dict)
    URZ_help = URZ.Utilities(env, RoutingZoo, episode, **kwargs)

    URZ_help.data()
    print("2", env.simulator.det_dict)

    URZ_help.Replace_data()
    env.reset()

env.close()

 Retrying in 1 seconds


1 {}
2 {}


KeyboardInterrupt: 

In [7]:
program = da.Table_record_creator(num_episodes,URZ_help.model,50,3)
program.table_record()

Run was succesful


,network,model,demand,Bounded,Greedy,link_value,link_std,TT_value,TT_std,entropy_value,entropy_std
0,grid,markow,1000,0.2,0,36.0,11.21,62985.0,4384.28,0.27,0.41


In [7]:
program = da.Table_record_creator(num_episodes,URZ_help.model,50,3)
program.table_record()

Run was succesful


,network,model,demand,Bounded,Greedy,link_value,link_std,TT_value,TT_std,entropy_value,entropy_std
0,grid2,markow,1000,0.2,0.3,28.0,10.58,62684.0,4444.18,0.73,0.2


In [7]:
program = da.Table_record_creator(num_episodes,URZ_help.model,50,3)
program.table_record()

Run was succesful


,network,model,demand,Bounded,Greedy,5_x,link_value,link_std,5_y,TT_value,TT_std,5,entropy_value,entropy_std
0,grid,1,markow,1000,0.2,0.3,28.0,10.58,0.3,62684.0,4444.18,0.3,0.73,0.2


In [8]:
program = da.Table_record_creator(num_episodes,URZ_help.model,50,3)
program.table_record()

Run was succesful


,network,model,demand,Bounded,Greedy,link_value,link_std,TT_value,TT_std,entropy_value,entropy_std
0,grid,markow,1000,0.2,0.3,11.0,4.1,62684.0,4444.18,0.73,0.2


In [ ]:
    def act(self, state):

        self.make_noise()

        utilities = list(map(lambda x,y: ((self.beta * x) + y), self.cost,self.stored_noises)) #RK: here to do: self.noise[k]

        if abs(self.old_c_hat[-2] - self.old_c_hat[-1]) > self.gamma_u:

            p = np.random.random()

            if p < self.greedy:

                action = np.random.randint(low=0,high=self.action_space) #RK: First bounded rationality, then greedy.

            else:

                action = np.argmin(utilities)
                self.stored_utilities = utilities
        
        else:

            action = np.argmin(self.stored_utilities)


        return action       


    def learn(self, action, observation):

        reward = self.get_reward(observation)
        self.last_reward = reward

        if abs(self.old_c_hat[-1]-reward) > self.gamma_c:

            if self.type == 'weight':
                c_hat = 0
                if self.cost[action] != reward: #RK: This is quite dangerous assumption. you beeter check this condition somehow else (what if the costs are the same on two paths?)
                    del(self.days[action][len(self.days[action])-1]) #RK: I would try to data structure called queue here https://docs.python.org/3/library/queue.html
                    self.days[action].insert(0,self.cost[action])
                for memory in range(self.remember): #memory

                    c_hat += self.alpha_j[memory] * self.days[action][memory]

            else:

                    c_hat = self.alpha_zero * self.cost[action] + self.alpha_j * reward

            self.cost[action] = c_hat
            self.old_c_hat.append(c_hat)